In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from pandas_datareader import data
from datetime import datetime, timedelta

%matplotlib inline

In [0]:
# Define the Stock and number of days as you prefer.
# Stock from https://finance.yahoo.com/
stock_name = 'AAPL'
day = 60

start_date = datetime(2018, 1, 1)
end_date = datetime(2019, 2, 28)

date_for_split_train_test = datetime(2018, 12, 31)

###################################################

In [0]:
# Preprocess data
stock = data.DataReader(stock_name, 'yahoo', start_date, end_date)

training_set = stock.reset_index()
training_set = training_set[training_set['Date'] <= date_for_split_train_test][['Date', 'Close']]

test_set = stock.reset_index()
test_set = test_set[test_set['Date'] > date_for_split_train_test][['Date', 'Close']]

test_set = test_set.reset_index()
test_set = test_set.drop(columns=['index'])

In [0]:
def feature_generate(data, day):
  feature = []

  for i in range(0, data.shape[0] - day):
    temp = data['Close'][i:i+day].values
    feature.append(temp)

  feature = np.asarray(feature)
  feature = np.reshape(feature, (feature.shape[0], feature.shape[1], 1))
  
  return feature

def target_generate(data, day):
  target = []

  for i in range(day, data.shape[0]):
    temp = data['Close'][i]
    target.append(temp)

  target = np.asarray(target)
  target = np.reshape(target, (target.shape[0], 1))
  
  return target

def test_generate(data, day):
  test = []
  
  for i in range(data.shape[0] - day, data.shape[0]):
    temp = data['Close'][i]
    test.append(temp)
    
  test = np.asarray(test)
  test = np.reshape(test, (test.shape[0], 1))
  
  return test  

In [0]:
x_train = feature_generate(training_set, day)
y_train = target_generate(training_set, day)

x_test = test_generate(training_set, day)
x_test_temp = np.reshape(x_test, (1, day, 1))

y_test = test_set['Close']

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [0]:
# Develop model with LSTM.
# You can edit this model.
model = keras.Sequential()

model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, return_sequences=True))
model.add(keras.layers.CuDNNLSTM(units=150, input_shape=(x_train.shape[1], 1)))


model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=100, batch_size=32)

In [0]:
model.evaluate(x_train, y_train)

In [0]:
# Plot the result
y_pred = []
x_test_copy = x_test

for i in range(0, y_test.shape[0]):
  x_test_temp = np.reshape(x_test_copy, (1, day, 1))
  temp = model.predict(x_test_temp)
  y_pred.append(temp[0][0])
  
  x_test_copy = np.append(x_test_copy, temp)[1:61]
  
y_pred = np.asarray(y_pred)

test_set['Predict'] = pd.DataFrame(y_pred, columns=['Predict'])

plt.figure(figsize=(16,10))
plt.title('Results')
plt.xlabel('Date')
plt.ylabel('Close')
plt.plot(test_set['Date'], test_set['Close'], 'b')
plt.plot(test_set['Date'], test_set['Predict'], 'r')
plt.xticks(rotation=60)
plt.show()

test_set = test_set.drop(columns=['Predict'])